In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

In [2]:
from urllib.parse import urlparse

In [3]:
@udf(stypes.StringType())
def get_url_path(url):
    try:
        url_parse = urlparse(url)
    except Exception as e:
        print(e)
        return None

    return url_parse.path

In [4]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

###### Load data 

In [9]:
begin_date = '2021-06-14'
end_date = '2021-06-20'

In [10]:
%%time
visit1 = hive.sql("""
 select cid,
        sbbolUserId,
        hitPagePath,
        hitPageTitle,
        sessionId,
        sessionDate,
        sessionStartTime,
        hitTime,
        hitType,
        eventCategory,
        eventAction,
        eventLabel,
        hitPageHostName
        
 from cap_external_google_dda_external_google_analytics.visit
where 1 = 1
  and sessiondate >= "{}"
  and sessiondate <= "{}"
  and (hitPageHostName = 'sbi.sberbank.ru'
       OR hitPagePath like '%/s_m_business%')
""".format(begin_date, end_date)).cache()

CPU times: user 12.4 ms, sys: 15.5 ms, total: 27.8 ms
Wall time: 1min 9s


In [11]:
drop_table(SBX_TEAM_DIGITCAMP, "tmp_visit_140621_200621",hive)

In [12]:
create_table_from_df(SBX_TEAM_DIGITCAMP, "tmp_visit_140621_200621", visit1, hive)

In [14]:
begin_date = '2021-07-11'
end_date = '2021-07-17'

In [15]:
%%time
visit2 = hive.sql("""
 select cid,
        sbbolUserId,
        hitPagePath,
        hitPageTitle,
        sessionId,
        sessionDate,
        sessionStartTime,
        hitTime,
        hitType,
        eventCategory,
        eventAction,
        eventLabel,
        hitPageHostName
        
 from cap_external_google_dda_external_google_analytics.visit
where 1 = 1
  and sessiondate >= "{}"
  and sessiondate <= "{}"
  and (hitPageHostName = 'sbi.sberbank.ru'
       OR hitPagePath like '%/s_m_business%')
""".format(begin_date, end_date)).cache()

CPU times: user 2.12 ms, sys: 1.02 ms, total: 3.13 ms
Wall time: 117 ms


In [16]:
drop_table(SBX_TEAM_DIGITCAMP, "tmp_visit_110721_170721",hive)

In [17]:
%%time
create_table_from_df(SBX_TEAM_DIGITCAMP, "tmp_visit_110721_170721", visit2, hive)

CPU times: user 294 ms, sys: 373 ms, total: 667 ms
Wall time: 30min 22s


In [18]:
load_table(SBX_TEAM_DIGITCAMP, "tmp_visit_110721_170721", hive).show()

,cid,sbbolUserId,hitPagePath,hitPageTitle,sessionId,sessionDate,sessionStartTime,hitTime,hitType,eventCategory,eventAction,eventLabel,hitPageHostName
0,1398391897.1623246858,3c25f0a4-5744-1d5b-e054-90e2ba9a7f6c,/accounts-payments/account/:accountId,СберБизнес - версия: 41.002.02_0004,1626265802,2021-07-14,1626265802,1435660,EVENT,[std]: accounts-payments,show,[accountsandpayments paymentstable]: [showed],sbi.sberbank.ru
1,77746169.1624874021,8bd16390-6646-4614-969c-12ebf078542f,/main,СберБизнес,1626280970,2021-07-14,1626280970,118183,EVENT,[std]: event feed,show,[event feed table]: [showed],sbi.sberbank.ru
2,1702179496.1576215983,3c25f0bb-e5f4-1d5b-e054-90e2ba9a7f6c,/login/check-otp,СберБизнес,1626266647,2021-07-14,1626266647,8728,EVENT,[operations]: login,login-check-otp,[error:none],sbi.sberbank.ru
3,1342472216.1525423709,3c25f0ba-aec2-1d5b-e054-90e2ba9a7f6c,/accounts-payments,СберБизнес,1626246822,2021-07-14,1626246822,1291076,EVENT,[operations]: offers,impression-story,[placement: accounts_and_payments_list[context]][container: b3a7c69f-907a-5450-e054-90e2ba8333f9][bc: adv_bank][product: e-acquiring][new-concept:[show: deployed]],sbi.sberbank.ru
4,1980586260.1591613696,3c25f0a1-530b-1d5b-e054-90e2ba9a7f6c,/accounts-payments#lightbox/create/budget-payment,СберБизнес,1626245713,2021-07-14,1626245713,2413174,EVENT,[std]: payments,click,[ruble_payment_edit]: [open spoiler],sbi.sberbank.ru
5,1050843820.1603650465,e981dd7f-e78a-4947-84b0-55c7f99216cd,/accounts-payments/orders/sign,СберБизнес,1626281521,2021-07-14,1626281521,131009,EVENT,[operations]: 0401060,sign-otp-check,rub_comm,sbi.sberbank.ru
6,1428151536.1490187898,3c25f0a6-5a0f-1d5b-e054-90e2ba9a7f6c,/accounts-payments/account/:accountId#lightbox/RUB_COMM/:id/edit,СберБизнес,1626256752,2021-07-14,1626256752,1400019,EVENT,[std]: nbs-spk-counterparty,show,"[sberrating widget]: [show popover details, color: blank]",sbi.sberbank.ru
7,1692427110.1598966621,ecce50aa-9c1f-453a-b6a2-7d43ec6e9487,/login,СберБизнес,1626245072,2021-07-14,1626245072,1,EVENT,[operations]: login,login,[presetlogin:disabled][error:none],sbi.sberbank.ru
8,685009989.1605022561,712dbfb1-934a-4fb7-a0aa-7b1ea46c2f69,/accounts-payments/create/budget-payment,СберБизнес,1626245141,2021-07-14,1626245141,717905,PAGE,None,None,None,sbi.sberbank.ru
9,1359062981.1581687121,93b6c0c4-05cd-4ea4-9b3f-04f1cdee4ac8,/main,СберБизнес,1626243259,2021-07-14,1626243259,969577,EVENT,[std]: event feed,show,[event feed table]: [showed],sbi.sberbank.ru


In [19]:
load_table(SBX_TEAM_DIGITCAMP, "tmp_visit_140621_200621", hive).show()

,cid,sbbolUserId,hitPagePath,hitPageTitle,sessionId,sessionDate,sessionStartTime,hitTime,hitType,eventCategory,eventAction,eventLabel,hitPageHostName
0,856298277.1623591106,3c25f084-5daa-fc84-e054-90e2ba85d0c8,/login/check-otp,СберБизнес,1624113388,2021-06-19,1624113388,82748,EVENT,[std]: login,show,[checkotpform]: [sms-auto-check-mode],sbi.sberbank.ru
1,1480243203.1622439204,b174dd55-c694-4737-b24d-e8b81bd3f9b3,/loginSuccess,СберБизнес,1624091459,2021-06-19,1624091459,45225,PAGE,None,None,None,sbi.sberbank.ru
2,1480243203.1622439204,b174dd55-c694-4737-b24d-e8b81bd3f9b3,/accounts-payments#lightbox/payment/:type/:id,СберБизнес,1624091459,2021-06-19,1624091459,186456,EVENT,[field]: payments,ruble_payment_edit__rub_comm,amount,sbi.sberbank.ru
3,608363848.1482924177,307407c2-5641-48a8-9f4e-24a0f7984ba6,/main,СберБизнес,1624083710,2021-06-19,1624083710,74543,EVENT,[operations]: offers,impression-story,[placement: main[main_feed]][container: 37a37f99-822a-4f08-b895-02dbe3e23728][bc: interaction][product: none],sbi.sberbank.ru
4,2138087980.1606649243,3c25f07a-feb8-fc84-e054-90e2ba85d0c8,/login/check-otp,СберБизнес,1624114713,2021-06-19,1624114713,33665,EVENT,[operations]: login,login-check-otp,[error:none],sbi.sberbank.ru
5,17533816.1623893847,faeccf1b-e733-4c38-b5b3-a01ed2e91e69,/main#modal__mandatory-overlay-group-modal,СберБизнес,1624098601,2021-06-19,1624098601,46518,EVENT,[std]: asup: placements,show,[mainfeedsbbol3]: [main feed offers images are shown first time],sbi.sberbank.ru
6,1987729676.1622574855,41ca1411-39ac-4652-adb8-7958e1a2c4eb,/login,СберБизнес,1624072952,2021-06-19,1624072952,65368,EVENT,[std]: login,show,[news]: [show],sbi.sberbank.ru
7,1541930505.1622549554,67706304-6a84-4171-8e78-118ac497cea0,/login/check-otp,СберБизнес,1624083322,2021-06-19,1624083322,33155,EVENT,[std]: login,show,[checkotpform]: [sms-auto-check-mode],sbi.sberbank.ru
8,822287901.1554204118,ae135d14-6598-453f-bc9f-60a7ef4f7945,/statements/daily,СберБизнес - версия: 41.001.00_0055,1624099331,2021-06-19,1624099331,105236,EVENT,[std]: accounts statements,click,[statements and reports page]: [account statement request form],sbi.sberbank.ru
9,1688505241.1617845434,3c25f074-f722-fc84-e054-90e2ba85d0c8,/accounts-payments/account/:accountId,СберБизнес - версия: 41.000.04_0005,1624088300,2021-06-19,1624088300,1509565,PAGE,None,None,None,sbi.sberbank.ru


In [ ]:
visit = hive.sql("""select cid,
                           sbbolUserId,
                           hitPagePath,
                           sessionStartTime,
                           hitTime,
                           hitPageHostName
                        
                     from cap_external_google_dda_external_google_analytics.visit
                     where 1 = 1
                       and sessiondate >= "{}"
                       and sessiondate <= "{}"
                       """.format(begin_date, end_date))

In [7]:
visit.cache()

DataFrame[cid: string, sbbolUserId: string, hitPagePath: string, sessionStartTime: bigint, hitTime: bigint, hitPageHostName: string]

In [8]:
visit.show()

,cid,sbbolUserId,hitPagePath,sessionStartTime,hitTime,hitPageHostName
0,646102924.1601918648,None,/PhizIC/private/payments/servicesPayments/edit.do?copying=true&recommendId=268016666,1624092640,367945,node4.online.sberbank.ru
1,1609114350.1624127217,None,/ru/about/strategy,1624112814,1336224,www.sberbank.com
2,965380456.1569269598,None,/PhizIC/confirm/way4.do,1624092886,37214,node1.online.sberbank.ru
3,1110311454.1608473088,None,/PhizIC/private/payments/template/view.do?id=671085740&history=false&archive=false,1624094070,618709,node1.online.sberbank.ru
4,778167023.1624132000,None,/loans/creditability,1624131965,59686,front.node0.online.sberbank.ru
5,1164984767.1620453475,None,/pfm/finances,1624089866,411774,web6.online.sberbank.ru
6,1665065124.1624118285,None,/PhizIC/private/payments/view.do?id=24335710798&history=true&archive=false&creationType=8,1624118285,121979,node4.online.sberbank.ru
7,285071983.1621248419,None,/PhizIC/private/payments/servicesPayments/edit.do?copying=true&debtId=AA08aa2aa5e5791740c172e4cc3d1392,1624112238,140800,node5.online.sberbank.ru
8,856298277.1623591106,3c25f084-5daa-fc84-e054-90e2ba85d0c8,/login/check-otp,1624113388,82748,sbi.sberbank.ru
9,1331083782.1419011242,None,/CSAFront/index.do#/,1624114718,6129,online.sberbank.ru


In [15]:
visit_sbbol = visit.select("cid",
                           "sbboluserid",
                           "sessionStartTime",
                           "hitTime",
                           "hitPageHostName",
                           get_url_path(f.col("hitPagePath")).alias("hitPagePath")) \
                   .distinct()\
                   .orderBy("sbboluserid", "sessionStartTime", "hitTime",  "hitPagePath")

In [16]:
visit_sbbol.cache()

DataFrame[cid: string, sbboluserid: string, sessionStartTime: bigint, hitTime: bigint, hitPageHostName: string, hitPagePath: string]

In [17]:
visit_sbbol.count()

686868777

In [18]:
visit_sbbol.show()

,cid,sbboluserid,sessionStartTime,hitTime,hitPageHostName,hitPagePath
0,665655226.1623617732,None,1623618000,0,lks.domclick.ru,/
1,719091194.1623617924,None,1623618000,0,osago.sberbank.ru,/
2,354588923.1623479990,None,1623618000,0,domclick.ru,/
3,1343171466.1585798623,None,1623618000,0,online.sberbank.ru,/
4,1558314699.1588027687,None,1623618000,0,sbermobile.ru,/
5,486355190.1623617593,None,1623618000,0,sberbusiness.live,/
6,1399304114.1623617836,None,1623618000,0,sberprime.sber.ru,/
7,240973433.1623617990,None,1623618000,0,promo.sber.ru,/12june
8,2136630026.1604649743,None,1623618000,0,promo.sber.ru,/12june/
9,1419604719.1623617782,None,1623618000,0,promo.sber.ru,/12june/


In [21]:
visit = load_table(SBX_TEAM_DIGITCAMP, "tmp_visit_140621_200621", hive)

In [22]:
visit.show()

,cid,sbboluserid,sessionStartTime,hitTime,hitPageHostName,hitPagePath
0,665655226.1623617732,None,1623618000,0,lks.domclick.ru,/
1,719091194.1623617924,None,1623618000,0,osago.sberbank.ru,/
2,354588923.1623479990,None,1623618000,0,domclick.ru,/
3,1343171466.1585798623,None,1623618000,0,online.sberbank.ru,/
4,1558314699.1588027687,None,1623618000,0,sbermobile.ru,/
5,486355190.1623617593,None,1623618000,0,sberbusiness.live,/
6,1399304114.1623617836,None,1623618000,0,sberprime.sber.ru,/
7,240973433.1623617990,None,1623618000,0,promo.sber.ru,/12june
8,2136630026.1604649743,None,1623618000,0,promo.sber.ru,/12june/
9,1419604719.1623617782,None,1623618000,0,promo.sber.ru,/12june/
